# 作業
Hint: 人物原始邊框座標 (60, 40), (420, 510)

請根據 Lena 圖做以下處理

對明亮度做直方圖均衡處理
水平鏡像 + 縮放處理 (0.5 倍)
畫出人物矩形邊框

In [17]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

img_path = 'D:\\Python\\1st DL\\Day005\\lena.png'

img = cv2.imread(img_path, cv2.IMREAD_COLOR)

# Hint: 矩形

In [3]:
img_hw = img.copy()
# 畫 矩形
# 圖片 左上座標 右下座標 BGR顏色 線的粗細程度
cv2.rectangle(img_hw, (60, 40), (420, 510), (0, 255, 0), -1)
# -1 -> 填上負值代表將矩形填滿
# opencv 畫圖是直接畫在圖片上面，不是回傳一個畫好圖的圖片

cv2.imshow('img', img_hw)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

# Hint: 線

In [4]:
img_hw = img.copy()
# 畫 線
# 圖片 左上座標 右下座標 BGR顏色 線的粗細程度
cv2.line(img_hw, (60, 40), (420, 510), (0, 255, 0), 3)

cv2.imshow('img', img_hw)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

# Hint: 文字

In [5]:
# 畫 文字
# 圖片 要加的文字 文字左下角的位置 字型 字體大小 字體顏色 字體粗細
# 字型連結 https://codeyarns.com/2015/03/11/fonts-in-opencv/
img_hw = img.copy()

cv2.putText(img_hw, 'I LOVE YOU', (60, 40), 7, 1, (255, 0, 0), 2)

cv2.imshow('img', img_hw)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

# 解法一
根據題目，參考之前的投影片，最簡單的方式是

顏色的操作 (對明亮度做直方圖均衡)
把矩形邊框畫在圖片上之後在做位置上的變換
得到的圖的結果正確，但你不知道新的矩型邊框座標點

In [28]:
point1 = (60, 40)
point2 = (420, 510)
img_hw = img.copy()

"""
對明亮度做直方圖均衡
"""

# BGR -> hls
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)

# 對明亮度做直方圖均衡
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1]) 

# 轉成bgr
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
畫人物矩形
"""
cv2.rectangle(img_hw, point1, point2, (0, 0, 255), 3)

# 水平鏡像 + 縮放處理(0.5倍)
img_hw = img_hw[:, ::-1, :]

# 縮放處理
img_hw = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

cv2.imshow('img_hw', img_hw)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

# 解法二
如果希望得知矩型邊框的位置

顏色的操作 (對明亮度做直方圖均衡)
鏡像可以透過四則運算得知
透過建構 transformation matrix 做縮放
把矩型邊框的點與 transformation matrix 相乘就會得到縮放後的位置
畫圖
得到的圖的結果正確，同時也知道新的矩型邊框座標點

In [27]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]

"""
對明亮度做直方圖均衡
"""

# 原始 BGR 圖片轉 HSV 圖片
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)

# 對明亮度做直方圖均衡 -> 對 HSV 的 V 做直方圖均衡
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])

# 將圖片轉回 BGR
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
水平鏡像
"""
h, w = img_hw.shape[:2]

# 圖片鏡像
img_hw = img_hw[:, ::-1, :]

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
point1[0] = w - point1[0]
point2[0] = w - point2[0]
point1[0], point2[0] = point2[0], point1[0]
print("point1: {}, point2: {}".format(point1, point2))

"""
縮放處理(0.5倍)
"""
fx = 0.5
fy = 0.5
resize_col = int(img_hw.shape[1] * fx)
resize_row = int(img_hw.shape[0] * fy)

# 建構scale matrix
M_scale = np.array([[fx, 0, 0],
                    [0, fy, 0]], dtype=np.float32)
print(M_scale)
img_hw = cv2.warpAffine(img_hw, M_scale, (resize_col, resize_row))

# 把左上跟右下轉為矩陣型式
bbox = np.array((point1, point2), dtype=np.float32)
print(bbox)

point_result = np.dot(M_scale.T, bbox)
point_result = point_result.astype("uint8")
print(point_result)
scale_point1 = tuple(point_result[0])
scale_point2 = tuple(point_result[1])
print("scale_point1 = {}, scale_point2 = {}".format(scale_point1, scale_point2))

cv2.rectangle(img_hw, scale_point1, scale_point2, (0, 0, 255), 3)
# 按下esc
while True:
    cv2.imshow('image', img_hw)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

point1: [92, 40], point2: [452, 510]
[[0.5 0.  0. ]
 [0.  0.5 0. ]]
[[ 92.  40.]
 [452. 510.]]
[[ 46  20]
 [226 255]
 [  0   0]]
scale_point1 = (46, 20), scale_point2 = (226, 255)


# 解法三 (optional)
把矩型邊框用遮罩的方式呈現，使用同樣處理圖片的方式處理遮罩 最後再找遮罩的左上跟右下的點的位置

這邊會用到許多沒提過的東西，所以當作 optional

In [47]:
img_hw = img.copy()

point1 = [60, 40]
point2 = [420, 510]

#建立圖形遮罩
mask = np.zeros(img_hw.shape[0:2], dtype="uint8")

"""
對明亮度做直方圖均衡
"""
# 原始 BGR 圖片轉 HSV 圖片
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)

# 對明亮度做直方圖均衡 -> 對 HSV 的 V 做直方圖均衡
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])

# 將圖片轉回 BGR
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

"""
水平鏡像
"""

h, w = img_hw.shape[:2]
print("h = {}, w = {}".format(h, w))
# 圖片鏡像
img_hw = img_hw[:, ::-1, :]

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
print("before point1: {}, point2: {}".format(point1, point2))
point1[0] = w - point1[0]
point2[0] = w - point2[0]
point1[0], point2[0] = point2[0], point1[0]
print("after point1: {}, point2: {}".format(point1, point2))
mask[point1[0]:point2[0], point1[0]:point2[0]] = 255

# mask[92:452, 92:452] = 255

img_hw = cv2.bitwise_and(img_hw, img_hw, mask = mask)

"""
進行縮放處理
"""
img_hw = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

# 按下esc
cv2.imshow('img_hw', img_hw)
# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()
# while True:
#     cv2.imshow('img_hw', img_hw)
#     k = cv2.waitKey(0)
#     if k == 27:
#         cv2.destroyAllWindows()
#         break

h = 512, w = 512
before point1: [60, 40], point2: [420, 510]
after point1: [92, 40], point2: [452, 510]
